### Day 11: Dumbo Octopus

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:

    5483143223
    2745854711
    5264556173
    6141336146
    6357385478
    4167524645
    2176841721
    6882881134
    4846848554
    5283751526

The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

First, the energy level of each octopus increases by 1.

Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)

Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

Before any steps:

    11111
    19991
    19191
    19991
    11111

After step 1:

    34543
    40004
    50005
    40004
    34543

After step 2:

    45654
    51115
    61116
    51115
    45654

An octopus is highlighted when it flashed during the given step.

After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

**General Approach**
- We have a grid of NxN octopi
- Each octopus increases its level every step
- When it reaches 9, the next step, it increases the level of adjacent octopi by 1. 
- Then that 9 goes back to 0.
- We will count the number of times any octopus or cell reaches 0 (known as a flash)

**Tools**
- Need to read the grid
- Need to create a stepping function to increase everyone by 1
- Need to have some form of function that for any cell that reaches 0 it increases the neighboring cell's count by 1

**Grid Structure**
- We have a grid with the existing values [X]
- We need to walk through the adjacent positions, and increase that value by 1 [X]
- Create a function that checks the status of the adjacent values.
- Create another grid that checks whether the value has been lit in this turn or not.
- Find a way to recursively iterate over the grid until there are no more flashes.
- Then if it reaches 9, we change it by a factor.

In [1]:
!ln -s /Users/andrescrucettanieto/Documents/GitHub/advent_of_code/common/util.py util.py

ln: util.py: File exists


In [6]:
import util
import os
import numpy as np

In [7]:
os.chdir('/Users/andrescrucettanieto/Documents/GitHub/advent_of_code/2021')

In [8]:
small_grid = util.read_grid('data/11_small.txt')
medium_grid = util.read_grid('data/11_large.txt')
full_grid = util.read_grid('data/11.txt')

In [125]:
def increase_adjacent(grid,position,lit_array):
    '''
    Increases the adjacent values of the given position by 1.
    This happens when the value at the given position is 9.
    '''
    x,y = position
    for dx,dy in grid.DIRECTIONS:
        try: 
            # If the value hasn't been lit before increase it.
            if not lit_array[x+dx,y+dy]:
                grid[x+dx,y+dy] += 1
        except IndexError:
            continue

In [126]:
# Creating a function to iterate through the grid and increase the value of each position
def step(grid):
    '''
    Performs a single step over the grid.
    '''    
    # Add 1 to all the values in the array.
    grid._grid += 1
    
    # Create array of values that have been lit
    lit_array = (grid._grid < 0)
    
    while True:
        flashed = False
        for y in range(grid.max_y):
            for x in range(grid.max_x):
                val = grid[x,y]
                # If the position is more than 9, increase the adjacent values
                if val > 9 and not lit_array[x,y]:
                    lit_array[x,y] = True
                    increase_adjacent(grid,(x,y),lit_array)
                    flashed = True
        if not flashed:
            break
                    
    # After all of the steps, the positions that have reached more than 9 will be set to 0.
    new_grid = np.where(grid._grid > 9, 0, grid._grid)
    
    # Count the number of positions that have reached 0.
    flashes = np.sum(new_grid == 0)
    grid = util.Grid(new_grid)
    
    return grid, flashes

In [127]:
def task1(grid,N):
    '''
    Count the number of flashes after N steps.
    '''
    total_flashes = 0
    for _ in range(N):
        grid,flashes = step(grid)
        total_flashes += flashes
    return total_flashes

In [137]:
def task2(grid):
    '''
    Iterates until all the grid has been lit on a given turn.
    '''
    steps = 0
    while True:
        steps += 1 
        grid,flashes = step(grid)
        rows,columns = grid._grid.shape
        required_flashes = rows*columns
        if flashes == required_flashes:
            return steps

In [138]:
# Creating a function to check output of the grid compared with actual results.
test_grid_stp2 = util.read_grid('data/11_med_actual.txt')

In [139]:
grid_sm = util.Grid(small_grid)
grid_med = util.Grid(medium_grid)
grid_full = util.Grid(full_grid)

In [134]:
grid_full._grid.shape

(10, 10)

In [129]:
task1(grid_full,100)

1773

In [140]:
task2(grid_full)

494